In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("../data/train/variants/fr/variants_hist_data.csv", sep =';')
df2 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv")
df2["time"] = pd.to_datetime(df2["time"])
df['dep'] = df['dep'].replace({'2A':'201','2B':'202'}).astype(int)
df = df[df['dep']<203]
#df = df.groupby(["dep","semaine"], level = -1).sum()["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]
df= df.groupby(['dep', 'semaine'])[["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]].sum().drop(columns = ["dep"]).reset_index()
df

,dep,semaine,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,1,2021-02-12-2021-02-18,686,42
1,1,2021-02-13-2021-02-19,706,46
2,1,2021-02-14-2021-02-20,730,46
3,1,2021-02-15-2021-02-21,748,48
4,1,2021-02-16-2021-02-22,852,52
...,...,...,...,...
4219,202,2021-03-23-2021-03-29,302,14
4220,202,2021-03-24-2021-03-30,302,18
4221,202,2021-03-25-2021-03-31,328,18
4222,202,2021-03-26-2021-04-01,318,16


In [3]:
datalist = []
referencedate = pd.to_datetime("2021-02-12")

In [4]:
#10 premiers caractères datetime
#réaffecter directement
#récupérer la date minimum de variant_list (=df avec datetime)

def to_datalist(row):
    date = pd.to_datetime(row["semaine"][0:10], yearfirst = True)
    datalist.append((date, row["dep"], row["Nb_susp_501Y_V1"],row["Nb_susp_501Y_V2_3"]))
    return None

In [5]:
#for i in tqdm(df.index):
#    date = pd.to_datetime(df.loc[i,"semaine"][0:10], yearfirst = True)
#    datalist.append((date, df.loc[i,"dep"], df.loc[i,"Nb_susp_501Y_V1"],df.loc[i,"Nb_susp_501Y_V2_3"]))

In [6]:
df.apply(to_datalist, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4219    None
4220    None
4221    None
4222    None
4223    None
Length: 4224, dtype: object

In [7]:
len(datalist), df.shape

(4224, (4224, 4))

In [8]:
datalistdf = pd.DataFrame(datalist)
datalistdf.columns=["semaine","dep", "Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]
datalistdf

,semaine,dep,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,2021-02-12,1,686,42
1,2021-02-13,1,706,46
2,2021-02-14,1,730,46
3,2021-02-15,1,748,48
4,2021-02-16,1,852,52
...,...,...,...,...
4219,2021-03-23,202,302,14
4220,2021-03-24,202,302,18
4221,2021-03-25,202,328,18
4222,2021-03-26,202,318,16


In [9]:
# regarder pour chaque row de Enriched si :
# - le jour est avant le jour de début des variants
# - le jour est dans la semaine suivant le row des variants
# - append à Enriched+variant (= df de enriched + infos sur les variants)

date2 = row['time'] - pd.Timedelta("30 days")
dates = pd.date_range(start = date2, periods=31).tolist()



NameError: name 'row' is not defined

In [10]:
datalist3 = []

def test (data_row):
    counter=0
    if data_row['dep']==depnum:
        date1 = data_row['semaine']
        if date<referencedate:
            datalist3.append((date, 0,0))
            counter +=1
            datefound= True
        elif ((date >= referencedate) & (date1 <= date) & (date < (date1 + pd.Timedelta("7 days")))):
            V1 = data_row['Nb_susp_501Y_V1']
            V2 = data_row['Nb_susp_501Y_V2_3']
            counter +=1
            datefound=True
        if counter==1:
            break
    return None


def to_datalist2(row):
    date = row['time']
    depnum = row['numero']
    datefound=False
    
    datalistdf.apply(test, axis=1)
    
    if datefound==False:
        datalist2.append((date, V1, V2))
    
    
    

SyntaxError: 'break' outside loop (<ipython-input-10-9414be9b346b>, line 17)

In [11]:
datalist2 = []
for i in tqdm(df2.index):
    date = df2.loc[i,"time"]
    depnum = df2.loc[i,"numero"]
    datefound = False

    for j in datalistdf.index:
        counter = 0
        if datalistdf.loc[j,"dep"]==depnum: 
            date1 = datalistdf.loc[j,"semaine"]
            if (date < referencedate):
                datalist2.append((date,0,0))
                counter +=1
                datefound = True

            elif ((date >= referencedate) & (date1 <= date) & (date < (date1 + pd.Timedelta("7 days")))):
                V1 = datalistdf.loc[j,"Nb_susp_501Y_V1"]
                V2 = datalistdf.loc[j,"Nb_susp_501Y_V2_3"]
                datalist2.append((date,V1,V2))
                counter += 1
                datefound = True
            if (counter == 1):
                break
    if datefound == False:
        datalist2.append((date, V1,V2))  
          

100%|██████████| 30590/30590 [32:04<00:00, 15.90it/s]  


In [12]:
dfvar = pd.DataFrame(datalist2)
dfvar.columns=["date", "V1","V2"]
dfvar

,date,V1,V2
0,2020-05-14,0,0
1,2020-05-15,0,0
2,2020-05-16,0,0
3,2020-05-17,0,0
4,2020-05-18,0,0
...,...,...,...
30585,2021-03-27,9561,432
30586,2021-03-28,9557,428
30587,2021-03-29,9496,396
30588,2021-03-30,8643,364


In [13]:
df2["Nb_susp_501Y_V1"]=dfvar["V1"]
df2["Nb_susp_501Y_V2_3"]=dfvar["V2"]

In [21]:
df2['time']

0       2020-05-14
1       2020-05-15
2       2020-05-16
3       2020-05-17
4       2020-05-18
           ...    
30585   2021-03-27
30586   2021-03-28
30587   2021-03-29
30588   2021-03-30
30589   2021-03-31
Name: time, Length: 30590, dtype: datetime64[ns]

In [35]:
cam = pd.read_csv('df2_cam.csv')

In [36]:
cam['time'] = cam['time'].apply(lambda x: pd.to_datetime(x))

In [37]:
test = cam[['time','Nb_susp_501Y_V1','Nb_susp_501Y_V2_3']] == df2[['time','Nb_susp_501Y_V1','Nb_susp_501Y_V2_3']]
test['Nb_susp_501Y_V2_3'].unique()

array([ True])